When I started blogging, I read that [Ghost](https://ghost.org/) was a great choice because it's fast to load, scales well, and has a great writing interface. 

After 5 years, I've paid $1.8k for hosting a blog that only gets 6-8k visits per month, writing it in an interface that I hate.

It doesn't sound like a great deal.

## Migration steps

1. Download the JSON dump of my blog.
![Exporting my blog](images/ghost-export.png)
2. Set up blog with [Quarto](https://quarto.org/).
3. Process JSON dump to convert posts to Quarto posts.

I used a Python script to convert the JSON dump to a set of Quarto posts.


In [ ]:
#| code-fold: true
#| code-summary: "Show the code"

import json
import os
from datetime import datetime

import requests
from bs4 import BeautifulSoup
from markdownify import markdownify as md


BLOG_URL = "https://dylancastillo.co"
BLOG_JSON_DUMP = "../dylan-castillo.ghost.2024-05-28-10-39-09.json"
BLOG_AUTHOR_NAME = "Dylan Castillo"


def download_images(markdown_content, post_slug):
    soup = BeautifulSoup(markdown_content, "html.parser")
    images = soup.find_all("img")
    if images:
        os.makedirs(post_slug, exist_ok=True)
        for img in images:
            img_url_raw = img["src"]
            img_url = img_url_raw.replace("__GHOST_URL__", BLOG_URL)
            img_name = os.path.basename(img_url)
            response = requests.get(img_url, stream=True)
            if response.status_code == 200:
                print(f"Downloading image: {img_url} to {post_slug}/{img_name}")
                with open(os.path.join(post_slug, img_name), "wb") as f:
                    f.write(response.content)
                markdown_content = markdown_content.replace(
                    img_url_raw, os.path.join(post_slug, img_name)
                )
            else:
                print(f"Failed to download image: {img_url}")
    return markdown_content


def process_posts(data):
    posts = data["db"][0]["data"]["posts"]
    for post in posts:
        print("Processing post:", post["title"])
        title = post["title"]
        description = post["custom_excerpt"]
        author = BLOG_AUTHOR_NAME
        date = (
            datetime.strptime(post["published_at"], "%Y-%m-%dT%H:%M:%S.%fZ").strftime(
                "%m/%d/%Y"
            )
            if post["published_at"]
            else ""
        )
        date_modified = (
            datetime.strptime(post["updated_at"], "%Y-%m-%dT%H:%M:%S.%fZ").strftime(
                "%m/%d/%Y"
            )
            if post["updated_at"]
            else ""
        )

        # Convert HTML content to Markdown
        markdown_content = download_images(
            post["html"] if post["html"] else "", post["slug"]
        )
        markdown_content = md(markdown_content, code_language="python")
        markdown_content = markdown_content.replace("__GHOST_URL__", BLOG_URL)
        markdown_content = f"""---\ntitle: "{title}"\ndescription: "{description}"\nauthor: "{author}"\ndate: "{date}"\ndate-modified: "{date_modified}"\n---\n\n{markdown_content}"""

        # Save the markdown content to a file
        filename = f"{post['slug']}.md"
        with open(filename, "w", encoding="utf-8") as file:
            file.write(markdown_content)


if __name__ == "__main__":
    with open(BLOG_JSON_DUMP) as file:
        data = json.load(file)
    process_posts(data)

4. Manual review of posts.
5. Set up GitHub pages with Quarto.
6. Publish new blog.